# Install necessary packages

In [ ]:
%pip install -r requirements.txt

# Part 2: Time Series Features & Tree-Based Models

**Objective:** Extract basic time-series features from heart rate data, train Random Forest and XGBoost models, and compare their performance.

## 1. Setup

Import necessary libraries.

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

## 2. Data Loading

Load the dataset.

In [2]:
def load_data(file_path):
    # check if the file exists
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"The file {file_path} does not exist. Please run generate_data.py first.")
    
    # load the CSV file using pandas and parse timestamps
    df = pd.read_csv(file_path, parse_dates=['timestamp'])
    
    # display basic information about the loaded data
    print(f"Loaded {df.shape[0]} records with {df.shape[1]} features.")
    print(f"Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")
    print(f"Number of unique patients: {df['patient_id'].nunique()}")
    print(f"Heart rate range: {df['heart_rate'].min()} to {df['heart_rate'].max()} bpm")
    
    return df

## 3. Feature Engineering

Implement `extract_rolling_features` to calculate rolling mean and standard deviation for the `heart_rate`.

In [3]:
def extract_rolling_features(df, window_size_seconds):
    df_result = df.copy()
    
    # sort data by patient_id and timestamp
    df_sorted = df_result.sort_values(['patient_id', 'timestamp'])
    
    # create empty columns for our rolling statistics
    df_sorted['hr_rolling_mean'] = np.nan
    df_sorted['hr_rolling_std'] = np.nan
    
    # process each patient separately to avoid mixing data between patients
    for patient_id in df_sorted['patient_id'].unique():
        patient_data = df_sorted[df_sorted['patient_id'] == patient_id].copy()
        
        # set timestamp as index
        patient_data = patient_data.set_index('timestamp')
        
        # calculate rolling mean and standard deviation
        rolling_window = patient_data['heart_rate'].rolling(window=f'{window_size_seconds}s')
        
        # calculate statistics on this window
        patient_data['hr_rolling_mean'] = rolling_window.mean()
        patient_data['hr_rolling_std'] = rolling_window.std()
        
        # reset index to bring timestamp back as a column
        patient_data = patient_data.reset_index()
        
        # update the values in the original dataframe
        idx = df_sorted['patient_id'] == patient_id
        df_sorted.loc[idx, 'hr_rolling_mean'] = patient_data['hr_rolling_mean'].values
        df_sorted.loc[idx, 'hr_rolling_std'] = patient_data['hr_rolling_std'].values
    
    # handle NaN values (from the start of each patient's time series)
    df_sorted['hr_rolling_mean'] = df_sorted.groupby('patient_id')['hr_rolling_mean'].fillna(method='ffill')
    df_sorted['hr_rolling_std'] = df_sorted.groupby('patient_id')['hr_rolling_std'].fillna(method='ffill')
    
    for patient_id in df_sorted['patient_id'].unique():
        patient_mask = df_sorted['patient_id'] == patient_id
        
        # calculate patient's overall mean and std for heart rate
        hr_mean = df_sorted.loc[patient_mask, 'heart_rate'].mean()
        hr_std = df_sorted.loc[patient_mask, 'heart_rate'].std()
        
        mean_mask = patient_mask & df_sorted['hr_rolling_mean'].isna()
        std_mask = patient_mask & df_sorted['hr_rolling_std'].isna()
        
        df_sorted.loc[mean_mask, 'hr_rolling_mean'] = hr_mean
        df_sorted.loc[std_mask, 'hr_rolling_std'] = hr_std if not np.isnan(hr_std) else 0
    
    # feature summary
    print(f"Generated rolling features with {window_size_seconds} second window:")
    print(f"  - hr_rolling_mean range: {df_sorted['hr_rolling_mean'].min():.1f} to {df_sorted['hr_rolling_mean'].max():.1f}")
    print(f"  - hr_rolling_std range: {df_sorted['hr_rolling_std'].min():.1f} to {df_sorted['hr_rolling_std'].max():.1f}")
    print(f"  Missing values after processing: {df_sorted[['hr_rolling_mean', 'hr_rolling_std']].isna().sum().sum()}")
    
    return df_sorted

## 4. Data Preparation

Implement `prepare_data_part2` using the newly engineered features.

In [4]:
def prepare_data_part2(df_with_features, test_size=0.2, random_state=42):
    # select relevant features including the rolling features
    selected_features = [
        'age', 'systolic_bp', 'diastolic_bp', 'glucose_level', 
        'bmi', 'heart_rate', 'hr_rolling_mean', 'hr_rolling_std'
    ]
    
    # add smoker_status as dummy variables if it exists in the dataframe
    if 'smoker_status' in df_with_features.columns:
        # convert smoker_status to dummy variables (one-hot encoding)
        smoker_dummies = pd.get_dummies(df_with_features['smoker_status'], prefix='smoker')
        
        # add the dummy columns to the dataframe
        df_with_features = pd.concat([df_with_features, smoker_dummies], axis=1)
        
        # add the dummy column names to the selected features
        selected_features.extend(smoker_dummies.columns.tolist())
    
    X = df_with_features[selected_features]
    
    # select target variable
    y = df_with_features['disease_outcome']
    
    # split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    print(f"Training set: {X_train.shape[0]} samples with {X_train.shape[1]} features")
    print(f"Testing set: {X_test.shape[0]} samples with {X_test.shape[1]} features")
    
    # handle missing values using SimpleImputer
    imputer = SimpleImputer(strategy='mean')
    X_train = pd.DataFrame(
        imputer.fit_transform(X_train),
        columns=X_train.columns,
        index=X_train.index
    )
    
    X_test = pd.DataFrame(
        imputer.transform(X_test),
        columns=X_test.columns,
        index=X_test.index
    )
    
    print("Missing values before imputation:")
    print(f"  Training set: {df_with_features[selected_features].iloc[X_train.index].isna().sum().sum()}")
    print(f"  Testing set: {df_with_features[selected_features].iloc[X_test.index].isna().sum().sum()}")
    print("Missing values after imputation:")
    print(f"  Training set: {X_train.isna().sum().sum()}")
    print(f"  Testing set: {X_test.isna().sum().sum()}")
    
    return X_train, X_test, y_train, y_test

## 5. Random Forest Model

Implement `train_random_forest`.

In [5]:
def train_random_forest(X_train, y_train, n_estimators=100, max_depth=10, random_state=42):
    # initialize the Random Forest classifier with specified parameters
    rf_model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        class_weight='balanced',  # handle class imbalance
        random_state=random_state,
        n_jobs=-1  # use all available cores for faster training
    )
    
    # train the model
    rf_model.fit(X_train, y_train)
    
    # print feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("Random Forest Model Trained Successfully")
    print(f"\nTop 5 important features:")
    for i, (feature, importance) in enumerate(zip(feature_importance['feature'].values[:5], 
                                                 feature_importance['importance'].values[:5])):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    return rf_model

## 6. XGBoost Model

Implement `train_xgboost`.

In [6]:
def train_xgboost(X_train, y_train, n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42):
    # initialize the XGBoost classifier with specified parameters
    xgb_model = xgb.XGBClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        random_state=random_state,
        use_label_encoder=False,  
        eval_metric='logloss',    
        scale_pos_weight=(len(y_train) - sum(y_train)) / sum(y_train) 
    )
    
    # train the model
    xgb_model.fit(X_train, y_train)
    
    # get feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': xgb_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("XGBoost Model Trained Successfully")
    print(f"\nTop 5 important features:")
    for i, (feature, importance) in enumerate(zip(feature_importance['feature'].values[:5], 
                                               feature_importance['importance'].values[:5])):
        print(f"{i+1}. {feature}: {importance:.4f}")
    
    return xgb_model

## 7. Model Comparison

Calculate and compare AUC scores for both models.

In [7]:
def compare_models(models, X_test, y_test):
    results = {}
    
    print("Model Comparison:")
    print("=" * 40)
    
    # calculate AUC for each model
    for name, model in models.items():
        # generate probability predictions
        y_prob = model.predict_proba(X_test)[:, 1]
        
        # calculate AUC score
        auc = roc_auc_score(y_test, y_prob)
        results[name] = auc
        
        print(f"{name} AUC: {auc:.4f}")
    
    best_model = max(results, key=results.get)
    
    # calculate improvement percentage
    model_names = list(results.keys())
    if len(model_names) >= 2:
        model1, model2 = model_names[0], model_names[1]
        improvement = abs(results[model1] - results[model2])
        percent_change = (improvement / min(results[model1], results[model2])) * 100
        
        print("\nComparison Summary:")
        print(f"Absolute difference in AUC: {improvement:.4f}")
        print(f"Percent improvement: {percent_change:.2f}%")
        print(f"Best performing model: {best_model} (AUC: {results[best_model]:.4f})")
        
        # feature impact insight
        if best_model == "Random Forest":
            print("\nInsight: Random Forest often performs better when there are complex interactions")
            print("between features that aren't explicitly engineered.")
        else:
            print("\nInsight: XGBoost often performs better when there's a clear gradient")
            print("structure in the data that benefits from sequential improvement.")
    
    return results

## 8. Save Results

Save the AUC scores to a text file.

In [8]:
def save_results(model_results, file_path='results/results_part2.txt'):
    # create 'results' directory if it doesn't exist
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    # format results as strings
    best_model = max(model_results, key=model_results.get)
    lines = [
        "MODEL COMPARISON RESULTS",
        "=" * 40
    ]
    
    # add each model's AUC score
    for model_name, auc_score in model_results.items():
        lines.append(f"{model_name} AUC: {auc_score:.4f}")
    
    # calculate improvement
    if len(model_results) >= 2:
        model_names = list(model_results.keys())
        model1, model2 = model_names[0], model_names[1]
        improvement = abs(model_results[model1] - model_results[model2])
        percent_change = (improvement / min(model_results[model1], model_results[model2])) * 100
        
        lines.extend([
            "\nCOMPARISON SUMMARY",
            "=" * 40,
            f"Absolute difference in AUC: {improvement:.4f}",
            f"Percent improvement: {percent_change:.2f}%",
            f"Best performing model: {best_model}"
        ])
        
        # add insights about time-series features
        lines.extend([
            "\nFEATURE ENGINEERING INSIGHTS",
            "=" * 40,
            "Adding time-series features (rolling mean and standard deviation)",
            "allowed the models to capture temporal patterns in heart rate data,",
            "potentially improving their predictive performance compared to",
            "models using only static features."
        ])
    
    # add timestamp
    lines.append(f"\nEvaluation completed on: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # write results to file
    with open(file_path, 'w') as f:
        f.write('\n'.join(lines))
    
    print(f"\nResults saved to {file_path}")

## 9. Main Execution

Run the complete workflow.

In [10]:
if __name__ == "__main__":
    # load data
    data_file = 'data/synthetic_health_data.csv'
    df = load_data(data_file)
    
    # extract rolling features
    window_size = 300  # 5 minutes in seconds
    df_with_features = extract_rolling_features(df, window_size)
    
    # prepare data
    X_train, X_test, y_train, y_test = prepare_data_part2(df_with_features)
    
    # train models
    rf_model = train_random_forest(X_train, y_train)
    xgb_model = train_xgboost(X_train, y_train)
    
    # compare models
    models = {
        "Random Forest": rf_model,
        "XGBoost": xgb_model
    }
    
    # use our compare_models function
    model_results = compare_models(models, X_test, y_test)
    
    # save results
    save_results(model_results)
    
    print("\nFeature Engineering Impact:")
    print("The addition of time-series features (hr_rolling_mean and hr_rolling_std)")
    print("helps the models capture temporal patterns in the heart rate data,")
    print("providing context about patient health trends over time rather than")
    print("just point-in-time measurements.")

Loaded 7326 records with 10 features.
Date range: 2023-01-01 00:00:00 to 2023-08-05 13:16:36.634428
Number of unique patients: 150
Heart rate range: 40.0 to 120.9524593747634 bpm


C:\Users\mihir\AppData\Local\Temp\ipykernel_27384\1213384138.py:34: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fillna instead
  df_sorted['hr_rolling_mean'] = df_sorted.groupby('patient_id')['hr_rolling_mean'].fillna(method='ffill')
C:\Users\mihir\AppData\Local\Temp\ipykernel_27384\1213384138.py:34: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sorted['hr_rolling_mean'] = df_sorted.groupby('patient_id')['hr_rolling_mean'].fillna(method='ffill')
C:\Users\mihir\AppData\Local\Temp\ipykernel_27384\1213384138.py:35: FutureWarning: SeriesGroupBy.fillna is deprecated and will be removed in a future version. Use obj.ffill() or obj.bfill() for forward or backward filling instead. If you want to fill with a single value, use Series.fill

Generated rolling features with 300 second window:
  - hr_rolling_mean range: 40.0 to 121.0
  - hr_rolling_std range: 1.0 to 14.7
  Missing values after processing: 0
Training set: 5860 samples with 11 features
Testing set: 1466 samples with 11 features
Missing values before imputation:
  Training set: 600
  Testing set: 135
Missing values after imputation:
  Training set: 0
  Testing set: 0
Random Forest Model Trained Successfully

Top 5 important features:
1. diastolic_bp: 0.2088
2. glucose_level: 0.1779
3. hr_rolling_std: 0.1528
4. age: 0.1126
5. hr_rolling_mean: 0.1049
XGBoost Model Trained Successfully

Top 5 important features:
1. hr_rolling_mean: 0.2229
2. diastolic_bp: 0.1944
3. glucose_level: 0.1290
4. heart_rate: 0.1170
5. smoker_former: 0.0555
Model Comparison:
Random Forest AUC: 0.9947
XGBoost AUC: 1.0000

Comparison Summary:
Absolute difference in AUC: 0.0053
Percent improvement: 0.53%
Best performing model: XGBoost (AUC: 1.0000)

Insight: XGBoost often performs better whe

c:\Users\mihir\OneDrive\Desktop\DataSci223\5-put-a-label-on-it-MihirUCSF\venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:20:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
